Parallel Collections
---------------------

Systems like Spark and Dask include "big data" collections with a small set of high-level primitives like `map`, `filter`, `groupby`, and `join`.  With these common patterns we can often handle computations that are more complex than map, but are still structured.

In this section we repeat the submit example using the PySpark and the Dask.Bag APIs, which both provide parallel operations on linear collections of arbitrary objects.


### Objectives

*  Use high-level `pyspark` and `dask.bag` to parallelize common non-map patterns

### Requirements

*  Dask.bag

*Note: the following exercises were designed to work with `dask 0.10.1`, you can check your installed version of `dask` with the following code*:

```
import dask
print(dask.__version__)
```

## Application

We again start with the following sequential code

```python
series = {}
for fn in filenames:   # Simple map over filenames
    series[fn] = pd.read_hdf(fn)['x']

results = {}

for a in filenames:    # Doubly nested loop over the same collection
    for b in filenames:  
        if a != b:     # Filter out bad elements
            results[a, b] = series[a].corr(series[b])  # Apply function

((a, b), corr) = max(results.items(), key=lambda kv: kv[1])  # Reduction
```

### Spark/Dask.bag methods

We can construct most of the above computation with the following Spark/Dask.bag methods:

*  `collection.map(function)`: apply function to each element in collection
*  `collection.product(collection)`: Create new collection with every pair of inputs
*  `collection.filter(predicate)`: Keep only elements of colleciton that match the predicate function
*  `collection.max()`: Compute maximum element

We use these briefly in isolated exercises and then combine them to rewrite the previous computation from the `submit` section.

### Dask.bag: Example API

In [ ]:
import dask.bag as db

In [ ]:
b = db.from_sequence(range(5))
b

In [ ]:
b.compute()  # Gather results back to local process

### `map`

In [ ]:
# Square each element

b.map(lambda x: x ** 2).compute()

In [ ]:
# Square each element and collect results

b.map(lambda x: x ** 2).compute()

In [ ]:
# Select only the even elements

b.filter(lambda x: x % 2 == 0).compute()

In [ ]:
# Cartesian product of each pair of elements in two sequences (or the same sequence in this case)

b.product(b).compute()

In [ ]:
# Chain operations to construct more complex computations

(b.map(lambda x: x ** 2)
  .product(b)
  .filter(lambda tup: tup[0] % 2 == 0)
  .compute())

### Exercise: Parallelize pairwise correlations with Dask.bag

To make this a bit easier we're just going to compute the maximum correlation and not try to keep track of the stocks that yielded this maximal result.

In [ ]:
from glob import glob
import os
import pandas as pd

filenames = sorted(glob(os.path.join('..', 'data', 'json', '*.h5')))  # ../data/json/*.json
filenames[:5]

In [ ]:
%%time

### Sequential Code

series = []
for fn in filenames:   # Simple map over filenames
    series.append(pd.read_hdf(fn)['close'])

results = []

for a in series:    # Doubly nested loop over the same collection
    for b in series:  
        if not (a == b).all():     # Filter out comparisons of the same series 
            results.append(a.corr(b))  # Apply function

result = max(results)

In [ ]:
%%time

### Parallel code

b = db.from_sequence(filenames)

# TODO



In [ ]:
result

In [ ]:
%load solutions/collections-2.py

In [ ]:
result

In [ ]:
%%time

import dask

result = corr.compute(get=dask.local.get_sync)

### PySpark solution for comparison

In [ ]:
%load solutions/collections-1.py

### Conclusion

*  Higher level collections include functions for common patterns
*  Move data to collection, construct lazy computation, trigger at the end
*  Used PySpark (`cartesian + map`) and Dask.bag (`product + map`) to handle nested for loop